# Server Test
> This is the code I am using to test the server. In this test, I will use 2 connections and attempt to make the join the same game.

In [38]:
import socket as s

## Connecting to server

I connect to the GO server on port 80(configured in docker).

In [39]:
conn = s.socket(s.AF_INET, s.SOCK_STREAM)
conn.connect(("127.0.0.1", 80))

## Sending packets

All packets must be suffixed with \n. This is a python implemtation of how to send them:

In [40]:
# Sends a packet, returns whatever socket.Send returns
def send_packet(content: bytes):
    global conn
    return conn.send(content + b'\n')


## Creating a game

I need to send a packet with 'c' as the first character to intialise a game:

In [41]:
send_packet(b'c')

2

In response, I should get a packet with the *game code*.

In [42]:
BUFFER_SIZE = 1024

def recieve_packet() -> bytes:
    return conn.recv(BUFFER_SIZE)

## Recieve the response from creating the game

In [43]:
recieve_packet()

b'ECannot create a game without having a name.\nC{"Ok":true,"Name":"ltwpvtjw"}\n'

In [44]:
packet = recieve_packet()

In [ ]:
trimmed = packet[1:]
import json
packetDict = json.loads(trimmed)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## User 2

I will attempt to join the game as another user

In [ ]:
conn2 = s.socket(s.AF_INET, s.SOCK_STREAM)
conn2.connect(("127.0.0.1", 80))

In [ ]:
conn2.send(b'j{"Name":"' + packetDict['Name'].encode() + b'"}\n')
conn2.recv(BUFFER_SIZE)

b'ECannot join a game when you do not have a name.\n'


## Playing the game
The two players should be able to share data with each other.

In [ ]:
conn.send(
    b'f{"X":1, "Y":1, "Dx":999, "Dy":999}\n'
)

36

In [ ]:
conn.recv(BUFFER_SIZE)

b'EGame not started yet\n'

subequent reads should have different values for player 1, becuase Dx and Dy move the player along like so

In [ ]:
import time
time.sleep(1)

conn2.send(
    b'f{"X":1, "Y":1, "Dx":0, "Dy":0}\n'
)
conn2.recv(BUFFER_SIZE)

b'EUser not in game\n'

## Error cases

In [ ]:
conn3 = s.socket(s.AF_INET, s.SOCK_STREAM)
conn3.connect(("127.0.0.1", 80))

### Joining full/running game
This should fail and return an error packet explaining why

In [ ]:
conn3.send(b'j{"Name":"' + packetDict['Name'].encode() + b'"}\n')
conn3.recv(BUFFER_SIZE)

b'ECannot join a game when you do not have a name.\n'

### Creating a game with 1 player and attempting to send a frame
This should fail and return an error packet explaining why

In [ ]:
conn3.send(b'c\n')
conn3.recv(BUFFER_SIZE)#i dont care what the game id is

conn3.send(
    b'f{"X":1, "Y":1, "Dx":0, "Dy":0}\n'
)
conn3.recv(BUFFER_SIZE)

b'EGame not started yet\n'